# 二、类的继承与多态

### 1. 继承和多态
### 2. 抽象和接口
### 3. 类属性的访问权限
### 4. 用property实现属性的读写
---
---
## 1. 继承和多态
### A) 所有类的模版都是type, 所有类最终都继承于object

In [1]:
class A:
    pass

print(A.__class__)        # 所有类的模版都是type

<class 'type'>


In [2]:
print(type.__class__)  

<class 'type'>


In [3]:
print(A.__bases__)        # object是最大基类

(<class 'object'>,)


In [4]:
print(type.__bases__)     

(<class 'object'>,)


### B) 继承的基本语法
super(child_class, child_object).parent_attribute(arg)  
super( )里面的参数不写也可以

In [20]:
class Father(object):
    eat = 'Father eat'
    def __init__(self, name, age):
        self.age = age
        self.name = name

class Son(Father):
    def __init__(self, name, age):
        super().__init__('father_name', age + 25)    #调用父类的init, 必须先写
                                                     #如果用Father.__init__则没有实例会被绑定
        self.name = name
        self.ag = age
        

print(Son.__bases__)

s = Son('son_name',13)
print(s.name)
print(s.eat)
print(s.age)

print(s.__dict__)                                   # name是同名属性，同名的name属性被覆盖

(<class '__main__.Father'>,)
son_name
Father eat
38
{'age': 38, 'name': 'son_name', 'ag': 13}


In [28]:
class D(object):
    def __init__(self):
        print("init D")

class B(D):
    def __init__(self):
        super(B, self).__init__()
        print("init B")

class A(B):
    def __init__(self):
        super(A, self).__init__()
        print("init A")

a = A()

init D
init B
init A


### C)多继承

In [30]:
import inspect

""" 这个继承关系是这样的：
             D
            / \
           B   C
            \ /
             A
"""

class D(object):
    def __init__(self):
        print("init D")

class B(D):
    def __init__(self):
        super(B, self).__init__()
        print("init B")

class C(D):
    def __init__(self):
        super(C, self).__init__()
        print("init C")

class A(B,C):
    def __init__(self):
        super(A, self).__init__()
        print("init A")

print(inspect.getmro(A))

a = A()

(<class '__main__.A'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.D'>, <class 'object'>)
init D
init C
init B
init A


### D)覆盖和重写

In [55]:
class BaseClass():
    a = 1
    
    def __init__(self, num):
        self.b = num
    
    @classmethod
    def test(cls):
        print("base.a=%d 输出父类的a"%cls.a)
        
    def get_b(self):
        return self.b
    
    def say_hi(self):
        print('base:', self)
    
 
class SubClass(BaseClass):
    a = 2
    
    def __init__(self, num):
        super().__init__(num + 1)                  #这句没用，同名对象变量b会被下面那句重写
        self.b = num
    
    @classmethod
    def test(cls):
        print('以下调用子类test')
        print("sub.a=%d 输出子类的a" % cls.a)    
        super().test()                             #因为super()里面的参数都是子类，所以输出的实际是子类的a                               
        print("调用基类的a %d\n" % super().a)        #只有这样输出的才是父类的a
                                                   #类变量这样才可以，对象变量只会被重写
    
    def get_sub_b(self):
        print("sub: ", self.b)
        print("base: ", super().get_b())
        
    def say_hi(self):
        super().say_hi()                          #这里虽然调用了父类的方法，但super传过去的self是子类
        print('sub: ', self)
        
 
SubClass.test()      

c = SubClass(5)
c.get_sub_b()
c.say_hi()                                        #只调用子类方法

以下调用子类test
sub.a=2 输出子类的a
base.a=2 输出父类的a
调用基类的a 1

sub:  5
base:  5
base: <__main__.SubClass object at 0x7ffec12b8d30>
sub:  <__main__.SubClass object at 0x7ffec12b8d30>


### E) 多态

In [69]:
class Person(object):
    def __init__(self, name, age):
        self.age = age
        self.name = name
        
    def say_hi(self):
        print('I am a', self.__class__.__name__, ', my name is ', self.name)
        
    def tell_age(self):
        print('My age is:', self.age)
        

class Female(Person):
    def __init__(self, name, age):
        super().__init__(name, age)

    def tell_age(self):
        print('My age is a secret, but I will tell you: ', self.age)
        
class Male(Person):
    def __init__(self, name, age):
        super().__init__(name, age)
        
    def tell_age(self):
        print('Ahaha, my age is', self.age, '. And you')


def say_something(ob):
    ob.say_hi()
    ob.tell_age()
    print()
        
p = Person('p', 10)
f = Female('f', 12)
m = Male('m', 20)

say_something(p)
say_something(f)
say_something(m)

I am a Person , my name is  p
My age is: 10

I am a Female , my name is  f
My age is a secret, but I will tell you:  12

I am a Male , my name is  m
Ahaha, my age is 20 . And you



## 2. 抽象和接口

抽象基类如下所示，metaclass=ABCMeta,不能被实例化

In [77]:
from abc import ABCMeta, abstractmethod

class Action(metaclass=ABCMeta):    #每个abstractmethod都必须被实现
    @abstractmethod
    def run(self):
        pass

    @abstractmethod
    def eat(self):
        pass
    
    def drink(self):
        print('Drink water')

class People(Action):
    def run(self):
        print('People run')
        
    def eat(self):
        print('People eat')
        


# a = Action()                    # Can't instantiate abstract class Action with abstract methods eat, run
p = People()
p.run()
p.eat()
p.drink()

People run
People eat
Drink water


## 3. 类属性的访问权限

In [96]:
class Father:
    x = 0
    _x = 1
    __x = 5
    
class Child(Father):
    def func1(self):
        print("Child, ", self._x)   
        
    def func2(self):
        print("Child, ", self.__x)   
        
    def func3(self):
        print("Child, ", super().__x)
        
f = Father()
print(f._x)                         
# print(f.__x)                         #'Father' object has no attribute '__x'

s = Child()
print(s._x)
# print(s.__x)                         #'Child' object has no attribute '__x'

s.func1()                              # 子类内可以访问 _x
# s.func2()                            # 'Child' object has no attribute '_Child__x'
# s.func3()                            # 'super' object has no attribute '_Child__x'

print(Father.__dict__)                 # __x 变成了 _Father__x
print(Child.__dict__) 

print(Father._Father__x)              # 改成这就ok了

1
1
Child,  1
{'__module__': '__main__', 'x': 0, '_x': 1, '_Father__x': 5, '__dict__': <attribute '__dict__' of 'Father' objects>, '__weakref__': <attribute '__weakref__' of 'Father' objects>, '__doc__': None}
{'__module__': '__main__', 'func1': <function Child.func1 at 0x7ffec1814ca0>, 'func2': <function Child.func2 at 0x7ffec1814d30>, 'func3': <function Child.func3 at 0x7ffec1814940>, '__doc__': None}
5


## 4.用property实现属性的读写

### A)  方法1：

In [98]:
class Person(object):
    def __init__(self, v):
        self.__age = v
        
    def get_age(self):
        print("get_age method")
        return self.__age
    
    def set_age(self, val):
        print("set_age method")
        self.__age = val
        
    the_age = property(get_age, set_age)   
    #这个the_age是一个描述器，用的时候就好像一个属性一样，实际上会调用相应的方法
    
p = Person(0)
p.the_age = 18
print(p.the_age)

set_age method
get_age method
18


### B) 方法2:

@property装饰器就是负责把一个方法变成属性调用的：把一个getter方法变成属性，只需要加上@property就可以了，此时，@property本身又创建了另一个装饰器@score.setter，负责把一个setter方法变成属性赋值

In [105]:
class Person(object):
    def __init__(self, name, age):
        self.__name = name
        self.__age = age
    
    @property
    def age(self):
        return self.__age
    
    @age.setter             #两个函数名和.setter那个必须相同       
    def age(self, val):
        self.__age = val
    
    @property
    def name(self):
        return self.__name
    
    @name.setter                    
    def name(self, val):
        self.__name = val
        
p = Person('haha',12)
print(p.name, p.age)

p.age = 18
p.name = 'heihei'
print(p.name, p.age)

haha 12
heihei 18
